In [1]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [127]:
import os, requests
from bs4 import BeautifulSoup

##Scrape

In [128]:
class Scraper:
    def __init__(self, parent_url):
        self.parent_url = parent_url
        self.all_urls = set()

    def get_all_urls(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a')
        urls = [link['href'] for link in links if link['href'].startswith(self.parent_url)]
        for u in urls:
            if u.startswith(self.parent_url) and u not in self.all_urls:
                self.get_all_urls(u)
        self.all_urls.update(urls)

    def scrape(self, url):
        page = requests.get(url)
        return BeautifulSoup(page.content, 'html.parser')

    def preprocess(self, soup):
        corpus = ""
        for s in soup.find_all(['header', 'title', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p']):
            corpus += str(s.text) + '\n'
        return corpus

In [129]:
def save_txt(protocol, doc, url):
    with open(f"{protocol}-documentation/{'_'.join(url.split('/'))}.txt", 'w') as f:
        f.write(doc)

In [130]:
protocol = 'lido'
parent_url = "https://docs.lido.fi/"
os.makedirs(f"{protocol}-documentation", exist_ok=True)

scraper = Scraper(parent_url)
scraper.get_all_urls(parent_url)
all_urls = scraper.all_urls
for u in all_urls:
    soup = scraper.scrape(u)
    doc = scraper.preprocess(soup)
    save_txt(protocol, doc, u)

In [131]:
all_urls

{'https://docs.lido.fi/contracts/lido-oracle/#add-calculation-of-staker-rewards-apr',
 'https://docs.lido.fi/guides/steth-integration-guide',
 'https://docs.lido.fi/token-guides/steth-on-aave-caveats'}